# Setup

## Import libraries

In [238]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression #, Lasso, Ridge, LassoCV,BayesianRidge
import statsmodels.formula.api as sm
#import matplotlib.pylab as plt

## you need to install dmba library.if you get error message about dmba, 
##please see week 1 "Getting Started with Python" file, 
##Installing dmba in Anaconda Prompt, pip install dmba  

from dmba import regressionSummary#, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import AIC_score #, BIC_score, adjusted_r2_score

## Load data

In [239]:
CHRoster_df_orig = pd.read_csv('CommunityHealthRoster_Demo.csv')
CHRoster_df_orig.head()

,Unnamed: 0,ProviderName,Age,Sex,Social_Risk_Factor,Num_Social_Risk_Factors,Inpatient_Psych_Service,Num_IP_Facility_Days,Total_Cost_IP_Facility_Days,Avg_Cost_IP_Facility_Days,Emergency_Dept_Service,Num_Emergency_Dept_Services,Avg_Cost_Emergency_Dept_Services,Alcohol_or_Drug_Diagnosis,Major_Depressive_Disorder_Diagnosis,Race_Black_Indigenous_PeopleOfColor,Married,Employed
0,1,Chewbacca,52,F,0,0,0,NaN,NaN,NaN,0,NaN,NaN,0,0,0.0,NaN,0
1,2,AhsokaTano,65,M,0,0,0,NaN,NaN,NaN,1,10.0,197.900,1,1,0.0,1.0,0
2,3,Omega,55,F,0,0,0,NaN,NaN,NaN,0,NaN,NaN,0,0,0.0,NaN,0
3,4,Chewbacca,46,F,1,1,0,NaN,NaN,NaN,0,NaN,NaN,1,0,0.0,NaN,0
4,5,AhsokaTano,58,F,0,0,0,NaN,NaN,NaN,1,4.0,152.595,0,0,0.0,NaN,0


# Method

## Select variables for analysis

In [240]:
selected_columns = ['ProviderName', 
                    'Age', 
                    'Sex', 
                    'Social_Risk_Factor', 
                    'Num_Social_Risk_Factors',
                    'Inpatient_Psych_Service',
                    'Avg_Cost_IP_Facility_Days',
                    'Emergency_Dept_Service',
                    'Alcohol_or_Drug_Diagnosis',
                    'Major_Depressive_Disorder_Diagnosis',
                    #'Race_Black_Indigenous_PeopleOfColor',
                    #'Employed'
                   ]

CHRoster_df = CHRoster_df_orig[selected_columns]

CHRoster_df.head()

,ProviderName,Age,Sex,Social_Risk_Factor,Num_Social_Risk_Factors,Inpatient_Psych_Service,Avg_Cost_IP_Facility_Days,Emergency_Dept_Service,Alcohol_or_Drug_Diagnosis,Major_Depressive_Disorder_Diagnosis
0,Chewbacca,52,F,0,0,0,NaN,0,0,0
1,AhsokaTano,65,M,0,0,0,NaN,1,1,1
2,Omega,55,F,0,0,0,NaN,0,0,0
3,Chewbacca,46,F,1,1,0,NaN,0,1,0
4,AhsokaTano,58,F,0,0,0,NaN,1,0,0


### Sample based on IP Services

Select only Inpatient_Psych_Service == 1 becasue we are interested in the effect on IP copst amongst those who actually did have an inpatient stay.

In [241]:
# Filter for Inpatient_Psych_Service == 1
CHRoster_df = CHRoster_df[CHRoster_df['Inpatient_Psych_Service'] == 1]

CHRoster_df.head()

,ProviderName,Age,Sex,Social_Risk_Factor,Num_Social_Risk_Factors,Inpatient_Psych_Service,Avg_Cost_IP_Facility_Days,Emergency_Dept_Service,Alcohol_or_Drug_Diagnosis,Major_Depressive_Disorder_Diagnosis
15,Omega,57,F,0,0,1,NaN,0,1,0
26,AhsokaTano,55,F,0,0,1,1212.339565,0,1,1
123,Yoda,42,M,1,1,1,739.870000,0,1,0
139,LukeSkywalker,37,F,0,0,1,655.210000,0,1,0
146,Omega,46,F,1,3,1,1212.338824,1,1,1


#### Remove NaN from Outcome Variable (Avg_Cost_IP_Facility_Days)

In [242]:
# Count the number of NaN values in 'Avg_Cost_IP_Facility_Days' column
nan_count = CHRoster_df['Avg_Cost_IP_Facility_Days'].isna().sum()

# Display the count of NaN values
print(f"Number of NaN values in 'Avg_Cost_IP_Facility_Days' Before: {nan_count}")

# Remove rows with NaN values in 'Inpatient_Psych_Service' column
CHRoster_df = CHRoster_df.dropna(subset=['Avg_Cost_IP_Facility_Days'])

# Count the number of NaN values in 'Avg_Cost_IP_Facility_Days' column
nan_count = CHRoster_df['Avg_Cost_IP_Facility_Days'].isna().sum()

# Display the count of NaN values
print(f"Number of NaN values in 'Avg_Cost_IP_Facility_Days' After: {nan_count}")

Number of NaN values in 'Avg_Cost_IP_Facility_Days' Before: 172
Number of NaN values in 'Avg_Cost_IP_Facility_Days' After: 0


## Data Wrangling

### Data Types

In [243]:
CHRoster_df.dtypes

ProviderName                            object
Age                                      int64
Sex                                     object
Social_Risk_Factor                       int64
Num_Social_Risk_Factors                  int64
Inpatient_Psych_Service                  int64
Avg_Cost_IP_Facility_Days              float64
Emergency_Dept_Service                   int64
Alcohol_or_Drug_Diagnosis                int64
Major_Depressive_Disorder_Diagnosis      int64
dtype: object

### Create age groups

In [244]:
# Filter cases aged 16 and older
CHRoster_df = CHRoster_df[CHRoster_df['Age'] >= 16]

# Define age groups and labels
age_bins = [16, 25, 35, 50, 65, float('inf')]
age_labels = ['16to25', '26to35', '36to50', '51to65', '65_and_Up']

# Create a new column 'AgeGroup' based on age groups
CHRoster_df['AgeGroup'] = pd.cut(CHRoster_df['Age'], bins=age_bins, labels=age_labels, right=False)

# Convert 'AgeGroup' column to object type
CHRoster_df['AgeGroup'] = CHRoster_df['AgeGroup'].astype('object')

CHRoster_df.dtypes

ProviderName                            object
Age                                      int64
Sex                                     object
Social_Risk_Factor                       int64
Num_Social_Risk_Factors                  int64
Inpatient_Psych_Service                  int64
Avg_Cost_IP_Facility_Days              float64
Emergency_Dept_Service                   int64
Alcohol_or_Drug_Diagnosis                int64
Major_Depressive_Disorder_Diagnosis      int64
AgeGroup                                object
dtype: object

### Create Social Risk Factors groups

In [245]:
# Count distinct values in 'Num_Social_Risk_Factors' column
distinct_values_count = CHRoster_df['Num_Social_Risk_Factors'].value_counts()

# Display the count of distinct values
print("Distinct Values Count in 'Num_Social_Risk_Factors':")
print(distinct_values_count)

Distinct Values Count in 'Num_Social_Risk_Factors':
Num_Social_Risk_Factors
0    527
1    268
2    128
3     69
4     36
5     17
6      7
7      1
Name: count, dtype: int64


In [246]:
# Define social risk factor groups and labels
risk_factor_bins = [0, 2, 4, 6, float('inf')]
risk_factor_labels = ['0to2', '3to4', '5to6', '7_and_Up']

# Create a new column 'SocialRiskGroup' based on risk factor groups
CHRoster_df['SocialRiskGroup'] = pd.cut(CHRoster_df['Num_Social_Risk_Factors'], bins=risk_factor_bins, labels=risk_factor_labels, right=False)

CHRoster_df.dtypes

ProviderName                             object
Age                                       int64
Sex                                      object
Social_Risk_Factor                        int64
Num_Social_Risk_Factors                   int64
Inpatient_Psych_Service                   int64
Avg_Cost_IP_Facility_Days               float64
Emergency_Dept_Service                    int64
Alcohol_or_Drug_Diagnosis                 int64
Major_Depressive_Disorder_Diagnosis       int64
AgeGroup                                 object
SocialRiskGroup                        category
dtype: object

## Forge data set

In [247]:
CHRoster_df.head()

,ProviderName,Age,Sex,Social_Risk_Factor,Num_Social_Risk_Factors,Inpatient_Psych_Service,Avg_Cost_IP_Facility_Days,Emergency_Dept_Service,Alcohol_or_Drug_Diagnosis,Major_Depressive_Disorder_Diagnosis,AgeGroup,SocialRiskGroup
26,AhsokaTano,55,F,0,0,1,1212.339565,0,1,1,51to65,0to2
123,Yoda,42,M,1,1,1,739.870000,0,1,0,36to50,0to2
139,LukeSkywalker,37,F,0,0,1,655.210000,0,1,0,36to50,0to2
146,Omega,46,F,1,3,1,1212.338824,1,1,1,36to50,3to4
266,AhsokaTano,37,F,0,0,1,655.210000,0,1,0,36to50,0to2


### Define and make dummy variables

In [248]:
# create a list containing predictors' name
predictors = ['Sex',
              'AgeGroup',
              'SocialRiskGroup',
              'Major_Depressive_Disorder_Diagnosis'
             ] 
print(predictors)

['Sex', 'AgeGroup', 'SocialRiskGroup', 'Major_Depressive_Disorder_Diagnosis']


In [249]:
# define outcome/target variable
outcome = 'Avg_Cost_IP_Facility_Days'
print(outcome)

Avg_Cost_IP_Facility_Days


In [250]:
# check data type of the predictors
#overview of pandas's data type https://pbpython.com/pandas_dtypes.html
CHRoster_df[predictors].dtypes 

Sex                                      object
AgeGroup                                 object
SocialRiskGroup                        category
Major_Depressive_Disorder_Diagnosis       int64
dtype: object

In [251]:
#get k-1 dummies out of k categorical levels by removing the first level
x = pd.get_dummies(CHRoster_df[predictors], drop_first=True)

x.dtypes

Major_Depressive_Disorder_Diagnosis    int64
Sex_M                                   bool
AgeGroup_26to35                         bool
AgeGroup_36to50                         bool
AgeGroup_51to65                         bool
AgeGroup_65_and_Up                      bool
SocialRiskGroup_3to4                    bool
SocialRiskGroup_5to6                    bool
SocialRiskGroup_7_and_Up                bool
dtype: object

In [252]:
y = CHRoster_df[outcome]
y.head()

26     1212.339565
123     739.870000
139     655.210000
146    1212.338824
266     655.210000
Name: Avg_Cost_IP_Facility_Days, dtype: float64

### Partition Data

In [253]:
# partition data; split the data training (60%) vs. validation (40%)
# random_state=1: Pass an int for reproducible output across multiple function calls
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.4, random_state=1)

train_x.head()

,Major_Depressive_Disorder_Diagnosis,Sex_M,AgeGroup_26to35,AgeGroup_36to50,AgeGroup_51to65,AgeGroup_65_and_Up,SocialRiskGroup_3to4,SocialRiskGroup_5to6,SocialRiskGroup_7_and_Up
48334,1,False,False,False,True,False,True,False,False
1945,1,False,False,True,False,False,True,False,False
3789,1,False,False,True,False,False,False,False,False
10026,1,True,False,False,True,False,False,False,False
49209,1,True,False,True,False,False,True,False,False


In [254]:
# check training and validation data sets
data={'Data Set':['train_x', 'valid_x', 'train_y', 'valid_y'], 'Shape':[train_x.shape, valid_x.shape, train_y.shape, valid_y.shape]}
df=pd.DataFrame(data)
df

,Data Set,Shape
0,train_x,"(631, 9)"
1,valid_x,"(422, 9)"
2,train_y,"(631,)"
3,valid_y,"(422,)"


## Modeling

### With Pandas

In [255]:
#build linear regression model using the training data
CHRoster_lm = LinearRegression()
CHRoster_lm.fit(train_x, train_y)

LinearRegression()

In [256]:
# print coefficients
print(pd.DataFrame({'Predictor': x.columns, 'Coefficient': CHRoster_lm.coef_}))

                             Predictor  Coefficient
0  Major_Depressive_Disorder_Diagnosis    61.906036
1                                Sex_M   -22.278988
2                      AgeGroup_26to35    47.792657
3                      AgeGroup_36to50    22.678281
4                      AgeGroup_51to65    71.192433
5                   AgeGroup_65_and_Up  -176.184264
6                 SocialRiskGroup_3to4   -47.436814
7                 SocialRiskGroup_5to6   -12.095099
8             SocialRiskGroup_7_and_Up   -29.278893


In [257]:
# Get the y intercept
CHRoster_lm.intercept_

867.8918308251044

In [259]:
# print performance measures (training data)
regressionSummary(train_y, CHRoster_lm.predict(train_x))


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 221.4398
            Mean Absolute Error (MAE) : 173.3165
          Mean Percentage Error (MPE) : -15.8803
Mean Absolute Percentage Error (MAPE) : 30.1875


In [260]:
# Predicted Avg Costs (and Errors) for 20 cases in validation set and summary predictive measures for entire validation set 
# Use predict() to make predictions on a new set
CHRoster_lm_pred = CHRoster_lm.predict(valid_x)
result = pd.DataFrame({'Predicted': CHRoster_lm_pred, 
                       'Actual': valid_y,
                       'Residual': valid_y - CHRoster_lm_pred})
result.head(20)

,Predicted,Actual,Residual
18544,955.311536,824.390000,-130.921536
13390,955.311536,1212.340000,257.028464
32297,868.291123,655.210000,-213.081123
16825,955.311536,127.010667,-828.300869
3476,890.570111,984.497273,93.927161
54089,915.684488,739.870000,-175.814488
57089,882.361052,569.307325,-313.053728
28876,977.590524,1249.078000,271.487476
17985,669.428579,836.400000,166.971421
57075,929.797867,750.000000,-179.797867


In [261]:
# Print performance measures (vaildation data)
regressionSummary(valid_y, CHRoster_lm_pred)


Regression statistics

                      Mean Error (ME) : -26.3517
       Root Mean Squared Error (RMSE) : 236.6875
            Mean Absolute Error (MAE) : 180.0706
          Mean Percentage Error (MPE) : -30.5036
Mean Absolute Percentage Error (MAPE) : 43.2124


### With Statmodels

In [262]:
train_df = train_x.join(train_y)
train_df.head()

,Major_Depressive_Disorder_Diagnosis,Sex_M,AgeGroup_26to35,AgeGroup_36to50,AgeGroup_51to65,AgeGroup_65_and_Up,SocialRiskGroup_3to4,SocialRiskGroup_5to6,SocialRiskGroup_7_and_Up,Avg_Cost_IP_Facility_Days
48334,1,False,False,False,True,False,True,False,False,938.410000
1945,1,False,False,True,False,False,True,False,False,1116.766364
3789,1,False,False,True,False,False,False,False,False,1212.338333
10026,1,True,False,False,True,False,False,False,False,1218.398750
49209,1,True,False,True,False,False,True,False,False,919.117500


In [263]:
predictors = train_x.columns
predictors

Index(['Major_Depressive_Disorder_Diagnosis', 'Sex_M', 'AgeGroup_26to35',
       'AgeGroup_36to50', 'AgeGroup_51to65', 'AgeGroup_65_and_Up',
       'SocialRiskGroup_3to4', 'SocialRiskGroup_5to6',
       'SocialRiskGroup_7_and_Up'],
      dtype='object')

In [264]:
# create the linear model formula
#string_name.join(iterable); returns a string concatenated with the elements of iterable
formula = 'Avg_Cost_IP_Facility_Days ~' + '+'.join(predictors)

# Specify the formula for the linear model
# formula = "Avg_Cost_IP_Facility_Days ~ Major_Depressive_Disorder_Diagnosis + Sex_M + AgeGroup_26-35 + AgeGroup_36-50 + AgeGroup_51-65 + AgeGroup_65_and_Up + Q('SocialRiskGroup_3to4') + Q('SocialRiskGroup_5to6') + Q('SocialRiskGroup_7_and_Up')"

formula

'Avg_Cost_IP_Facility_Days ~Major_Depressive_Disorder_Diagnosis+Sex_M+AgeGroup_26to35+AgeGroup_36to50+AgeGroup_51to65+AgeGroup_65_and_Up+SocialRiskGroup_3to4+SocialRiskGroup_5to6+SocialRiskGroup_7_and_Up'

In [265]:
# Build the linear model
CHRoster_sm_lm = sm.ols(formula=formula, data=train_df).fit()

# Display the summary of the linear model
print(CHRoster_sm_lm.summary())

                                OLS Regression Results                               
Dep. Variable:     Avg_Cost_IP_Facility_Days   R-squared:                       0.049
Model:                                   OLS   Adj. R-squared:                  0.035
Method:                        Least Squares   F-statistic:                     3.568
Date:                       Mon, 26 Feb 2024   Prob (F-statistic):           0.000245
Time:                               15:51:56   Log-Likelihood:                -4302.8
No. Observations:                        631   AIC:                             8626.
Df Residuals:                            621   BIC:                             8670.
Df Model:                                  9                                         
Covariance Type:                   nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [267]:
## check model's accuracy on the training and validation data set
## Training
regressionSummary(train_y, CHRoster_sm_lm.predict (train_x))


Regression statistics

                      Mean Error (ME) : -0.0000
       Root Mean Squared Error (RMSE) : 221.4398
            Mean Absolute Error (MAE) : 173.3165
          Mean Percentage Error (MPE) : -15.8803
Mean Absolute Percentage Error (MAPE) : 30.1875


In [268]:
## Validation
CHRoster_sm_lm_pred_stat = CHRoster_sm_lm.predict(valid_x)

# print performance measures (validation data)
regressionSummary(valid_y, CHRoster_sm_lm_pred_stat)


Regression statistics

                      Mean Error (ME) : -26.3517
       Root Mean Squared Error (RMSE) : 236.6875
            Mean Absolute Error (MAE) : 180.0706
          Mean Percentage Error (MPE) : -30.5036
Mean Absolute Percentage Error (MAPE) : 43.2124


### Backward elimination for reducing predictors in CHRoster example

In [269]:
def train_model(variables):
    model = LinearRegression()
    model.fit(train_x[variables], train_y)
    return model
def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_x[variables]), model)

allVariables = train_x.columns
allVariables

Index(['Major_Depressive_Disorder_Diagnosis', 'Sex_M', 'AgeGroup_26to35',
       'AgeGroup_36to50', 'AgeGroup_51to65', 'AgeGroup_65_and_Up',
       'SocialRiskGroup_3to4', 'SocialRiskGroup_5to6',
       'SocialRiskGroup_7_and_Up'],
      dtype='object')

In [270]:
# backward_elimination is from dmba library provided by the textbook
best_model, best_variables = backward_elimination(allVariables, train_model, score_model, verbose=True)

best_variables

Variables: Major_Depressive_Disorder_Diagnosis, Sex_M, AgeGroup_26to35, AgeGroup_36to50, AgeGroup_51to65, AgeGroup_65_and_Up, SocialRiskGroup_3to4, SocialRiskGroup_5to6, SocialRiskGroup_7_and_Up
Start: score=8627.69
Step: score=8625.78, remove SocialRiskGroup_5to6
Step: score=8623.88, remove SocialRiskGroup_7_and_Up
Step: score=8622.78, remove AgeGroup_36to50
Step: score=8622.42, remove Sex_M
Step: score=8622.42, remove None


['Major_Depressive_Disorder_Diagnosis',
 'AgeGroup_26to35',
 'AgeGroup_51to65',
 'AgeGroup_65_and_Up',
 'SocialRiskGroup_3to4']

In [271]:
print(pd.DataFrame({'Predictor': best_variables, 'Coefficient': best_model.coef_}))

                             Predictor  Coefficient
0  Major_Depressive_Disorder_Diagnosis    59.337637
1                      AgeGroup_26to35    33.790982
2                      AgeGroup_51to65    56.280130
3                   AgeGroup_65_and_Up  -197.166619
4                 SocialRiskGroup_3to4   -48.536672


In [272]:
best_model.intercept_

870.4346178048928

In [273]:
regressionSummary(valid_y, best_model.predict(valid_x[best_variables]))


Regression statistics

                      Mean Error (ME) : -24.5149
       Root Mean Squared Error (RMSE) : 237.5805
            Mean Absolute Error (MAE) : 180.1430
          Mean Percentage Error (MPE) : -30.6612
Mean Absolute Percentage Error (MAPE) : 43.5376
